# Fashion MNIST

## Validate Azure ML SDK installation and get version number for debugging purposes

In [ ]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace
Initialize a workspace object from persisted configuration.

In [ ]:
# Initialize Workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print("Resource group: ", ws.resource_group)
print("Location: ", ws.location)
print("Workspace name: ", ws.name)

## Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [ ]:
import os
import shutil

project_folder = '../projects/fashion_mnist_azbai'
os.makedirs(project_folder, exist_ok=True)

shutil.copy('./scripts/train_Fashion_MNIST.py', project_folder)

## Create An Experiment
**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'fashion-mnist'
experiment = Experiment(workspace = ws, name = experiment_name)

## Create Azure Batch AI cluster (GPU-enabled) as a compute target

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_target_name = 'myazbai'

try:
    batch_ai_compute = AmlCompute(workspace=ws, name=compute_target_name)
    print('found existing:', batch_ai_compute.name)
except ComputeTargetException:
    print('creating new.')
    batch_ai_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_NC6",
        vm_priority="dedicated",
        min_nodes = 0,
        max_nodes = 4
    )
    batch_ai_compute = AmlCompute.create(
        ws, 
        name=compute_target_name, 
        provisioning_configuration=batch_ai_config
    )
    batch_ai_compute.wait_for_completion(show_output=True)

## Create a TensorFlow estimator
The AML SDK's TensorFlow estimator enables you to easily submit TensorFlow training jobs for both single-node and distributed runs. For more information on the TensorFlow estimator, refer [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-tensorflow).

In [ ]:
from azureml.train.dnn import TensorFlow

estimator = TensorFlow(source_directory=project_folder,
                       compute_target=batch_ai_compute,
                       entry_script='train_Fashion_MNIST.py',
                       node_count=1,
                       worker_count=1,
                       parameter_server_count=1,
                       conda_packages=['keras', 'matplotlib'],
                       use_gpu=True)

## Submit the Experiment
Finally, run the training job on the ACI

In [ ]:
run = experiment.submit(estimator)
run.tag("Description","Batch AI trained Fashion MNIST model")

In [ ]:
run

In [ ]:
run.wait_for_completion(show_output=True)

## Show Metrics

In [ ]:
# get all metris logged in the run
run.get_metrics()
metrics = run.get_metrics()

import numpy as np
print('loss is {0:.2f}, and accuracy is {1:0.2f}'.format(
    metrics['loss'], 
    metrics['accuracy']
))

In [ ]:
# Plot data to see relationships in training and validation data
import numpy as np
import matplotlib.pyplot as plt
epoch_list = list(range(1, len(metrics['Training Accuracy']) + 1))  # values for x axis [1, 2, ..., # of epochs]
plt.plot(epoch_list, metrics['Training Accuracy'], epoch_list, metrics['Validation Accuracy'])
plt.legend(('Training Accuracy', 'Validation Accuracy'))
plt.show()

## Download Model

In [ ]:
# show all the files stored within the run record
run.get_file_names()

In [ ]:
import os
import shutil

os.makedirs('../outputs', exist_ok=True)
run.download_file('outputs/saved_model.tar.gz', output_file_path='../outputs')
shutil.unpack_archive('../outputs/saved_model.tar.gz', './models/fashion_mnist/' + run.id.split('_')[1])
os.remove(path='../outputs/saved_model.tar.gz')